# Create labels and test deployed API

This notebook allows you to specify the labels and test the deployed API

Now we create a label file for our clusters with inuitive search terms that will help our users easily find this 
data. If using your own data, find meaningful labels that faciliate how your users will search for this data.

Here the dictionary key relates to the cluster id, and multiple labels can be applied to each cluster

In [ ]:
import pickle

label_file_name = "labels.pkl"

# 0 is the cluster id
dict = {2 : ['book cover', 'closed book'], 1 : ['open book', 'double spread'], 0: ['book shelf', 'library']}
f = open(f"../custom-skills-deployment/models/{label_file_name}","wb")  # We are saving the file to our models directory
pickle.dump(dict,f)
f.close()

## Test the deployed inference API Web App 

Once the labels are created, we can deploy the app. Refer back to README.md and then come back here to test the API once deployed.

In [ ]:
from dotenv import load_dotenv
import os

## NOTE: This notebook loads parameters from the .env file in the notebooks folder.
##       This are normally generated by the deployment script - see README.md

load_dotenv(dotenv_path='.env')

WEB_APP_KEY = os.getenv("WEB_APP_KEY")   # Set the KEY value you deployed your Web App with [YourSecretKeyCanBeAnything]
WEB_APP_URL = os.getenv("WEB_APP_URL")   # This is the deployed web app format: URL https://[appname].azurewebsites.net

SEARCH_API_KEY = os.getenv("SEARCH_API_KEY")  # Your ACS API Key
ACS_URL        = os.getenv("ACS_URL")         # Your ACS URL format https://[your ACS instance].search.windows.net
DATA_SOURCE    = os.getenv("DATA_SOURCE")     # The name for your data source, e.g. clustering-datasource

SKILLSET_NAME = os.getenv("SKILLSET_NAME")  # The name of your skillset, e.g. testclusteringskillset
COGSVC_KEY    = os.getenv("COGSVC_KEY")     # This is your Cognitive Services key that resides in the same region as ACS (used to compare custom vision captions and object detection)


INDEX_NAME = os.getenv("INDEX_NAME")  # The name for the index, e.g.  clustering-index

STORAGE_CONNECTION_STRING = os.getenv("STORAGE_CONNECTION_STRING") # The connection string for the storage account with the images
STORAGE_CONTAINER_NAME = os.getenv("STORAGE_CONTAINER_NAME") # The name of the container with the images


In [ ]:
from requests import post, put
import json

result_response = ""

headers = {
    "Ocp-Apim-Subscription-Key": WEB_APP_KEY,     
    "Content-Type": "application/json",

}

with open("../data/postman_request.json") as f:
    text = json.load(f)
    


try:
    url = f"{WEB_APP_URL}/api/extraction"
    resp = post(url=url, json=text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your KEY value is the same as on the web app setting KEY")
        
    if len(result_response['values'][0]['data']['label']) > 0:
        print(f"API deployed successfully - predicted label is {result_response['values'][0]['data']['label']}")

except Exception as e:
    print('Exception calling Web App', e)

result_response